In [ ]:
import numpy as np
import scipy.io
from midiutil import MIDIFile
from utils import *

In [ ]:
key_time = 50e-3*0.5
Fs = 48000

#keys = scipy.io.loadmat('MATLAB_data/31Jan_jc_upper_octave_noise.mat')['key']
keys = scipy.io.loadmat('MATLAB_data/jw_key.mat')['key']
notes_list = scipy.io.loadmat('notes_100.mat')['notes']
freq_list = scipy.io.loadmat('freq_480_octave_down.mat')['f_cropped']
# notes_list = scipy.io.loadmat('notes_88.mat')['notes']
# freq_list = scipy.io.loadmat('freq_480.mat')['f_cropped']
#freq_list = scipy.io.loadmat('freq_441.mat')['f_cropped']
notes_list = notes_list.reshape(np.size(notes_list))
freq_list = freq_list.reshape(np.size(freq_list))

note_num = 100 #88+12, 12 from upper octave

In [ ]:
intensity_map = scipy.io.loadmat('MATLAB_data/jw_int.mat')['filtered_int_db']

In [ ]:
def intensity2sign(intensity):
    thresholds = [35,55,75]
    if intensity < thresholds[0]:
        return 'p'
    elif intensity < thresholds[1]:
        return 'mp'
    elif intensity < thresholds[2]:
        return 'mf'
    else:
        return 'f'

In [ ]:
def picknotes(cur_key, cur_freq, cur_int, notes, notes_list):
    j = 0
    while j < len(cur_key):
        if cur_key[j] == 0:
            j += 1
            continue
        if cur_key[j] > 88:
            pitch = cur_key[j] - 12
        else:
            pitch = cur_key[j]
        start = j
        ori_int = cur_int[j]
        while cur_key[j] != 0 and j < len(cur_key)-1:
            j += 1
            ori_int = max(ori_int,cur_int[j])
        
        #intensity = get_intensity(cur_key[j-1],cur_freq,notes_list) * ori_int
        intensity = ori_int
        #intensity = get_intensity(cur_key[j-1],cur_freq,notes_list)
        intensity_sign = intensity2sign(intensity)
        #intensity_sign = intensity2sign(intensity)
        notes.append(Note(pitch, start, j - 1, intensity, intensity_sign))
        j += 1
    return notes

In [ ]:
notes = []
for i in range(np.shape(keys)[0]):
    cur_key = keys[i]
    cur_int = intensity_map[i]
    cur_freq = freq_list[i]
    notes = picknotes(cur_key, cur_freq, cur_int, notes, notes_list)

print(len(notes))

In [ ]:
notes = sorted(notes, key = lambda note: note.start)
new_notes = remove_repetitive(notes,note_num)
# pprint(new_notes)
# print(len(new_notes))

In [ ]:
nnew_notes = multichannel(new_notes,note_num)

In [ ]:
mf = MIDIFile(1)     # only 1 track
track = 0   # the only track
time = 0    # start at the beginning
mf.addTrackName(track, time, "Sample Track")
mf.addTempo(track, time, 1200)

In [ ]:
for note in nnew_notes:
    #v = int(20+note.sign()*20)
    # v = int(50+note.sign()*12.5)
    v = int(note.intensity)
    mf.addNote(track,note.channel,note.pitch+20, note.start, note.end-note.start+1,v)

In [ ]:
with open("midifile/jw_upper_octave_int_orix.mid", 'wb') as outf:
   mf.writeFile(outf)

In [ ]:
# plot_notes(new_notes)

In [ ]:
# def picknotes_old(cur_key,cur_freq):
#     j = 0
#     while j < len(cur_key):
#         if cur_key[j] != 0:
            
#             d = {}
#             d['note'] = cur_key[j]
#             d['start'] = j
#             while cur_key[j] != 0 and j < len(cur_key)-1:
#                 j += 1
#             d['end'] = j - 1
#             intensity = get_intensity(cur_key[j-1],cur_freq,notes_list[cur_key[j-1]-1])
#             d['intensity'] = intensity
#             notes.append(d)
#         j += 1